In [5]:
## parameters  key and bpm have a least 20% infulence over the songs streams
# predict key and bpm regargding all the other parameters
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from datetime import date

data = pd.read_csv("MAIN_DATASET_spotify2023.csv")

data.head()
encoder = LabelEncoder()
data["key"] = encoder.fit_transform(data["key"])
data["mode"] = encoder.fit_transform(data["mode"])
released_day_of_year = []

for i in range(len(data.index)):
    day_of_year = date(data['released_year'][i],
                    data['released_month'][i],
                    data['released_day'][i]).timetuple().tm_yday
    released_day_of_year.append(day_of_year)

# add new series to dataframe
data.insert(len(data.columns), "released_day_of_year", released_day_of_year, True)

data = data[["released_day_of_year", "streams",'bpm','key', 'mode', 'danceability_%', 'valence_%', 'energy_%','acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']]

for i in range(len(data)):
    data["streams"][i] = data["streams"][i] / 1000000
#data_standarized
data

,released_day_of_year,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,195,141.381703,125,2,0,80,89,83,31,0,8,4
1,82,133.716286,92,3,0,71,61,74,7,0,10,4
2,181,140.003974,138,7,0,51,32,53,17,0,31,6
3,235,800.840817,170,0,0,55,58,72,11,0,11,15
4,138,303.236322,144,0,1,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...
947,307,91.473363,144,0,0,60,24,39,57,0,8,3
948,294,121.871870,166,8,0,42,7,24,83,1,12,6
949,307,73.513683,92,3,0,80,81,67,4,0,8,6
950,293,133.895612,97,3,0,82,67,77,8,0,12,5


In [6]:
X = data.drop("streams", axis = 1)
y = data["streams"]

X_train, X_help, y_train, y_help = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_help, y_help, test_size=0.5, random_state=42)


val_percent = (len(X_val)/(len(data)))*100
test_percent = (len(X_test)/(len(data)))*100
train_percent = (len(X_train)/(len(data)))*100
print(f"Percent of Train: {train_percent:.2f} ,Validation:{val_percent:.2f} and Test dataset:{test_percent:.2f} ")


Percent of Train: 59.98 ,Validation:19.96 and Test dataset:20.06 


## Task 1,2

In [19]:
reg = LinearRegression()
weights_train = np.ones_like(y_train)
np.random.seed(42)
random_weights = np.random.rand(len(y_train))

reg.fit(X_train, y_train, sample_weight=weights_train)
y_pred_train = reg.predict(X_train)
y_pred = reg.predict(X_val)

rmse = np.sqrt(np.mean((y_pred_train - y_train)**2))

mean = mean_squared_error(y_train, y_pred_train)
print(f"Rmse: {rmse}, Mean:  {mean}")

Rmse: 553.6714135282607, Mean:  306552.0341583822


##  Task3 neural regresor

In [33]:
reg = MLPRegressor(hidden_layer_sizes=(5, 6, 5), random_state=50, max_iter=50, solver='sgd')
reg.fit(X_train, y_train)
y_pred_train = reg.predict(X_train)
y_pred = reg.predict(X_val)

# and then try some customization, for example:
#reg = MLPRegressor(hidden_layer_sizes=(15, 10, 5), random_state=1, max_iter=1000, solver='adam')

#hidden_layer_sizes=(5, 100, 500) 75% "sgd" 81%
#max_iter 11 best 83%

rmse = np.sqrt(np.mean((y_pred_train - y_train)**2))
mean = mean_squared_error(y_train, y_pred_train)
print(f"Rmse: {rmse}, Mean:  {mean}")

Rmse: 4736.018432041637, Mean:  22429870.58863813


## Task 4  Polynomial reggresor

In [11]:
# Create polynomial features
poly_features = PolynomialFeatures(degree = 3) #4 poziom najlepszy
X_train_poly = poly_features.fit_transform(X_train)

reg = LinearRegression()
reg.fit(X_train_poly, y_train)

y_pred_train1 = reg.predict(X_train_poly)

rmse = np.sqrt(np.mean((y_pred_train1 - y_train)**2))
mean = mean_squared_error(y_train, y_pred_train1)
print(f"Rmse: {rmse}, Mean:  {mean}")


Rmse: 356.11984534494525, Mean:  126821.34424850772


## Task 5 Logistic regressor

In [35]:
clf = LogisticRegression()

clf.fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
y_pred = clf.predict(X_val)

acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)

rmse = np.sqrt(np.mean((y_pred - y_val)**2))
mean = mean_squared_error(y_val, y_pred)
#print(f"Rmse: {rmse}, Mean:  {mean}")
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")

ValueError: Unknown label type: 'continuous'

# task 6 diffrent features

In [41]:
#data_1 = X_train[['bpm','key']]
#data_2 = X_train.drop(['bpm','key'], axis = 1)
#data_2
X_train_1 = X_train[[ 'bpm','key']]
X_val_1 = X_val[[ 'bpm','key']]
X_train_2 = X_train.drop(['bpm','key'], axis = 1)
X_val_2 = X_val.drop(['bpm','key'], axis = 1)
y_train

665     185.408548
272     104.992946
285      61.105704
360      70.069745
354     380.726517
          ...     
106    1605.224506
270     185.240616
860     164.856284
435     181.831132
102     172.825906
Name: streams, Length: 571, dtype: float64

## 1)

In [39]:

clf = LogisticRegression()

clf.fit(X_train_1, y_train)
y_pred_train = clf.predict(X_train_1)
y_pred = clf.predict(X_val_1)


acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)

rmse = np.sqrt(np.mean((y_pred - y_val)**2))
mean = mean_squared_error(y_val, y_pred)
#print(f"Rmse: {rmse}, Mean:  {mean}")
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")

ValueError: Unknown label type: 'continuous'

## 2)

In [42]:
clf = LogisticRegression()

clf.fit(X_train_2, y_train)
y_pred_train = clf.predict(X_train_2)
y_pred = clf.predict(X_val_2)

acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)

rmse = np.sqrt(np.mean((y_pred - y_val)**2))
mean = mean_squared_error(y_val, y_pred)
#print(f"Rmse: {rmse}, Mean:  {mean}")
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")
#czemu rmse i mean są inne  i co oznaczają

ValueError: Unknown label type: 'continuous'

# task 7

In [4]:
data = pd.read_csv("winequality-red.csv", sep  = ";")
X = data.drop("quality", axis = 1)
y = data["quality"]

X_train, X_help, y_train, y_help = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_help, y_help, test_size=0.33, random_state=42)

In [44]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

## SVM 

In [45]:
# Kernels: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} 
#linear the best but the longest so most complicated
clf_SVM = svm.SVC(kernel='linear', C = 2, max_iter = 50000 )

clf_SVM.fit(X_train, y_train)
y_pred_train = clf_SVM.predict(X_train)
y_pred = clf_SVM.predict(X_val)

acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")

ValueError: Unknown label type: 'continuous'

## MLP

In [25]:
#hidden_layer_sizes=(10, 5), random_state=1, max_iter=1000, solver='lbfgs'
clf_mlp = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=1500,random_state=6, solver='adam', alpha =0.001)

clf_mlp.fit(X_train, y_train)
y_pred_train = clf_mlp.predict(X_train)
y_pred = clf_mlp.predict(X_val)


acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")

Accuracy train: 0.6246648793565683, accuracy validation: 0.5545171339563862


## KNN

In [135]:
#dla distance accuracy 100%, dla knn mniejszych lepszy wynik # leaf nic nie robi
clf_KNN = KNeighborsClassifier(n_neighbors=2, weights = "uniform", algorithm = "ball_tree", p = 10, leaf_size = 10000)

clf_KNN.fit(X_train, y_train)
y_pred_train = clf_KNN.predict(X_train)
y_pred = clf_KNN.predict(X_val)

acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")

Accuracy train: 0.773011617515639, accuracy validation: 0.4766355140186916


## Decision Tree

In [19]:
clf_DT = tree.DecisionTreeClassifier(max_depth=3)

clf_DT.fit(X_train, y_train)
y_pred_train = clf_DT.predict(X_train)
y_pred = clf_DT.predict(X_val)

acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")

Accuracy train: 0.5996425379803396, accuracy validation: 0.5327102803738317


## Pipeline

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
clf_mlp1 = Pipeline([("scaler", StandardScaler()), ("mlp", MLPClassifier(max_iter=5000, random_state=1, solver='adam', alpha =0.0001))])
clf_mlp1.fit(X_train, y_train)

y_pred_train = clf_mlp1.predict(X_train)
y_pred = clf_mlp1.predict(X_val)


acc_train = accuracy_score(y_train, y_pred_train)
acc_val = accuracy_score(y_val, y_pred)
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")


Accuracy train: 0.9803395889186773, accuracy validation: 0.6448598130841121


In [ ]:
X_combined = pd.concat([X_val, X_train], ignore_index=True)
y_combined = pd.concat([y_val, y_train], ignore_index=True)

#best for wine: 
clf_mlp = MLPClassifier(hidden_layer_sizes = (40,20),max_iter=4500,random_state=4, solver='lbfgs', alpha =0.001)

clf_mlp.fit(X_combined, y_combined)
y_pred_train = clf_mlp.predict(X_combined)
y_pred = clf_mlp.predict(X_test)


acc_train = accuracy_score(y_combined, y_pred_train)
acc_val = accuracy_score(y_test, y_pred)
print(f"Accuracy train: {acc_train}, accuracy validation: {acc_val}")